In [1]:
%matplotlib inline
import os,random

os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["KERAS_FLAGS"]  = "device=gpu%d"%(0)

import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
from keras.utils import normalize
import matplotlib.pyplot as plt
import seaborn as sns
import cPickle, random, sys, keras

Using TensorFlow backend.


In [2]:
# Load the dataset ...
#  You will need to seperately download or generate this file
Xd = cPickle.load(open("RML2016.10a_dict.dat",'rb'))
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)

In [3]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
train_idx = np.random.choice(range(0,int(n_examples)), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(map(lambda x: mods.index(lbl[x][0]), train_idx))
Y_test = to_onehot(map(lambda x: mods.index(lbl[x][0]), test_idx))

In [4]:
# Convert Samples to images.
from scipy import signal
print X_train.shape
# widths = np.arange(1, 31)
# cwtmatr = signal.cwt(sig, signal.ricker, widths)
# plt.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
#            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())
# plt.show()

(110000, 2, 128)


In [11]:
def create_image_from_signal(x):
    widths = np.arange(1, 224+1)    
    r = normalize(signal.cwt(x[0], signal.ricker, widths),1)+0.5
    g = normalize(signal.cwt(x[1], signal.ricker, widths),1)+0.5
    b = (r + g)/ 2    
    rgb = np.dstack((r,g,b))
    return rgb

testSignal = create_image_from_signal(X[0])

(224, 128, 3)
[[[ 0.44727526  0.52958653  0.48843089]
  [ 0.39398342  0.52975811  0.46187077]
  [ 0.49953637  0.37892     0.43922819]
  ..., 
  [ 0.41697507  0.48541526  0.45119516]
  [ 0.44566056  0.52340646  0.48453351]
  [ 0.59420458  0.4458351   0.52001984]]

 [[ 0.43561952  0.50814548  0.4718825 ]
  [ 0.45317038  0.45699889  0.45508464]
  [ 0.52528676  0.42859355  0.47694016]
  ..., 
  [ 0.50436811  0.44633208  0.47535009]
  [ 0.51559142  0.44621114  0.48090128]
  [ 0.5401302   0.46298662  0.50155841]]

 [[ 0.46982066  0.48195618  0.47588842]
  [ 0.47071625  0.44471309  0.45771467]
  [ 0.47013735  0.41510437  0.44262086]
  ..., 
  [ 0.51503661  0.42811608  0.47157635]
  [ 0.53289408  0.42383101  0.47836254]
  [ 0.53991199  0.43564192  0.48777695]]

 ..., 
 [[ 0.59101254  0.40976611  0.50038932]
  [ 0.60128824  0.41704302  0.50916563]
  [ 0.54984461  0.4149079   0.48237625]
  ..., 
  [ 0.38449004  0.51241396  0.448452  ]
  [ 0.37193193  0.51257737  0.44225465]
  [ 0.37402079  0.499